## Use plain attributes instead of get and set methods
Programmers coming from other languages naturally try to implement getter and setter methods.

In [3]:
class OldResister(object):
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms
        
resister = OldResister(1000)
resister.get_ohms()

1000

In [4]:
resister.set_ohms(1500)
resister.get_ohms()

1500

You should always start implementations with simple public attributes

In [6]:
class Resister(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        
r1 = Resister(50e3)
r1.ohms

50000.0

Later, if you decide that you need special behavior when an attribute is set, you can migrate to the @property decorator:

In [10]:
class VoltageResistance(Resister):
    def __init__(self, ohms):
        super(VoltageResistance, self).__init__(ohms)
        self.voltage = 0
        
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms
        
    def __str__(self):
        return "Voltage = %d, Current = %0.2f" % (self.voltage, self.current)
        
r2 = VoltageResistance(1e3)
print(r2)

Voltage = 0, Current = 0.00


In [11]:
r2.voltage = 10
print(r2)

Voltage = 10, Current = 0.01


Specifying a setter also lets you perform type checking and validation on values passed to your class.

## Consider @property instead of refactoring attributes
One advanced but common use of @property is transitioning what was once a simple numerical attribute into an on the fly calculation.

In [2]:
class Test(object):

    def __init__(self, a, b):
        self.a = a
        self.b = b

    @property
    def c(self):
        return self.a + self.b
    
test = Test(2, 3)
test.c

5

## Use descriptors for reuseable @property methods
The descriptor protocol defines how attribute access is interpreted by the language. Descriptors are objects with any of __get__, __set__, __delete__.

In [4]:
# from the docs at https://docs.python.org/2/howto/descriptor.html
class RevealAccess(object):
    """A data descriptor that sets and returns values
       normally and prints a message logging their access.
    """

    def __init__(self, initval=None, name='var'):
        self.val = initval
        self.name = name

    def __get__(self, obj, objtype):
        print 'Retrieving', self.name
        return self.val

    def __set__(self, obj, val):
        print 'Updating', self.name
        self.val = val
        
class MyClass(object):
    x = RevealAccess(10, 'var "x"')
    y = 5
    

In [6]:
m = MyClass()
m.x

Retrieving var "x"


10

In [7]:
m.x = 20

Updating var "x"


In [8]:
m.x

Retrieving var "x"


20